Input - Val & test preds from single models, validation set, sample_submission CSV
- https://www.kaggle.com/priteshshrivastava/ieee-pipeline-2-a-model-a-catboost-feat-sel
- https://www.kaggle.com/priteshshrivastava/ieee-pipeline-2-b-model-b-random-forest
- https://www.kaggle.com/priteshshrivastava/ieee-pipeline-2-c-model-c-xgboost

Output - Competition submission file

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model
import os
import math
from sklearn.metrics import roc_auc_score

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/__notebook__.ipynb
/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/__output__.json
/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/custom.css
/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/__results__.html
/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/predsC.csv
/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/test_predsC.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/__notebook__.ipynb
/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/__output__.json
/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/test_predsB.csv
/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/custom.css
/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/__results__.h

In [2]:
#predsA = pd.read_csv("")
#test_predsA = pd.read_csv("")

predsB = pd.read_csv("/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/predsB.csv")
test_predsB = pd.read_csv("/kaggle/input/ieee-pipeline-2-b-model-b-random-forest/test_predsB.csv")

predsC = pd.read_csv("/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/predsC.csv")
test_predsC = pd.read_csv("/kaggle/input/ieee-pipeline-2-c-model-c-xgboost/test_predsC.csv")

predsB.head()

,0
0,0
1,0
2,0
3,0
4,0


### Defining function to calculate the evaluation metric

In [3]:
def auc(x,y): 
    return roc_auc_score(x,y)
def print_score(m):
    res = [auc(m.predict(train_X), train_y), auc(m.predict(val_X), val_y)]
    print(res)

### Form a new dataset for validation & test by stacking the predictions

In [4]:
stacked_predictions = np.column_stack((predsB, predsC))
stacked_predictions = pd.DataFrame({'predsB': stacked_predictions[:, 0],
                                    'predsC': stacked_predictions[:, 1]})

stacked_test_predictions = np.column_stack((test_predsB, test_predsC))
stacked_test_predictions = pd.DataFrame({'test_predsB': stacked_test_predictions[:, 0],
                                         'test_predsC': stacked_test_predictions[:, 1]})

stacked_predictions.head()

,predsB,predsC
0,0.0,0.002640
1,0.0,0.005781
2,0.0,0.001102
3,0.0,0.064677
4,0.0,0.000574


### Specify meta model & fit it on stacked validation set predictions

In [5]:
meta_model = linear_model.LogisticRegression()

In [6]:
val_y = pd.read_csv("/kaggle/input/ieee-pipeline-1-create-validation-set/val_y.csv")
val_y.head()

,isFraud
0,0
1,0
2,0
3,0
4,0


In [7]:
meta_model.fit(stacked_predictions, val_y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Use meta model to make preditions on the stacked predictions of test set

In [8]:
#final_predictions = meta_model.predict(stacked_test_predictions)  ## Or Lower AUC even though individual models score higher
final_predictions = meta_model.predict_proba(stacked_test_predictions)[:,1]   ## Probabilities generally imporove AUC

### Submit predictions

In [9]:
submission = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv')
submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [10]:
submission['isFraud'] = final_predictions 
submission.to_csv('meta_stacking_v1.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 